## synthetic data generation

In [2]:
import json
import random
from datetime import datetime, timedelta
import uuid

def generate_iot_data(num_devices=1000, num_entries_per_device=10):
    devices = []
    base_time = datetime.utcnow()
    
    for device_id in range(1, num_devices + 1):
        device_name = f"iot_device_{device_id:03d}"
        device_type = random.choice(["sensor", "gateway", "actuator", "controller"])
        location = random.choice(["factory_floor", "warehouse", "office", "outdoor"])
        
        entries = []
        for i in range(num_entries_per_device):
            timestamp = (base_time - timedelta(minutes=random.randint(0, 1440))).isoformat() + "Z"
            
            entry = {
                "event_id": str(uuid.uuid4()),
                "timestamp": timestamp,
                "device_id": device_name,
                "device_type": device_type,
                "location": location,
                "metrics": {
                    "temperature_c": round(random.uniform(15.0, 45.0), 2),
                    "network_latency_ms": random.randint(5, 500),
                    "vibration_mg": round(random.uniform(0.1, 10.0), 3),
                    "battery_level": random.randint(10, 100),
                    "signal_strength": random.randint(-90, -30)
                },
                "status": random.choice(["online", "offline", "degraded"]),
                "firmware_version": f"v{random.randint(1, 3)}.{random.randint(0, 9)}.{random.randint(0, 9)}"
            }
            entries.append(entry)
        
        device_data = {
            "device_id": device_name,
            "device_type": device_type,
            "location": location,
            "events": entries
        }
        devices.append(device_data)
    
    return {"iot_devices": devices}

out=generate_iot_data()

C:\Users\GenAIHYDSYPUSR108\AppData\Local\Temp\ipykernel_18224\3186128406.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  base_time = datetime.utcnow()


In [3]:
import pandas as pd
k=pd.DataFrame(out['iot_devices'])
k

,device_id,device_type,location,events
0,iot_device_001,actuator,outdoor,[{'event_id': '8716b6f5-efe4-4fa8-9e85-cbd0706...
1,iot_device_002,sensor,factory_floor,[{'event_id': '547dc07b-e282-4de6-bcfb-d9e8839...
2,iot_device_003,sensor,factory_floor,[{'event_id': '1f8b1e68-7ed3-4493-b81d-c3fb050...
3,iot_device_004,controller,outdoor,[{'event_id': '1c231189-ba2f-44b0-87a7-6d9e534...
4,iot_device_005,sensor,warehouse,[{'event_id': '4d8ee788-2a54-440c-9fbc-497792c...
...,...,...,...,...
995,iot_device_996,actuator,factory_floor,[{'event_id': '9390171a-3c49-42db-ace4-301e7bc...
996,iot_device_997,sensor,office,[{'event_id': '4993767c-6388-4572-945f-92d1f03...
997,iot_device_998,actuator,office,[{'event_id': '1c674282-7dcb-41d3-ab92-b99acc0...
998,iot_device_999,gateway,outdoor,[{'event_id': 'b95118a6-8c70-4187-b5bd-68ddb56...


In [4]:
from collections import defaultdict
devices=list(k['device_id'])
hash_map=defaultdict(str)

for ind,value in enumerate(devices):
    hash_map[value]=ind

In [17]:
k.iloc[0]

device_id                                         iot_device_001
device_type                                             actuator
location                                                 outdoor
events         [{'event_id': '8716b6f5-efe4-4fa8-9e85-cbd0706...
Name: 0, dtype: object

In [21]:
from langchain_openai import ChatOpenAI  
from langchain_core.prompts import PromptTemplate


import os  
import httpx  
client = httpx.Client(verify=False) 
llm = ChatOpenAI( 
base_url="https://genailab.tcs.in",
model = "azure_ai/genailab-maas-DeepSeek-R1", 
api_key="sk-psJWnUf2HGUE90ew2RGCyA",
# Will be provided during event.  And this key is for 
# Hackathon purposes only and should not be used for any unauthorized 
# purposes
temperature=0,
http_client = client
)

template = """
You are a helpful assistant that going to summarize the device events and display all the metrics and also give proper suggestion need to follow.

Context: {question}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)
inp_device=input('Enter your device ID: ')
formated_prompt=prompt.format(question=str(k.iloc[hash_map[inp_device]]['events']))

out_put=llm.invoke(formated_prompt)

Enter your device ID:  iot_device_998


In [22]:
print(out_put.content)



### Device Event Summary for **iot_device_998** (Office Actuator)  
**Total Events:** 10 (Sorted Chronologically)  

---

#### **Event Details**  
1. **2025-11-18 06:31:27**  
   - **Offline** | Firmware: v2.6.0  
   - **Metrics:**  
     - Temperature: 36.96°C  
     - Network Latency: 98 ms  
     - Vibration: 0.283 mg  
     - Battery: 52%  
     - Signal: -88 dBm  

2. **2025-11-18 09:48:27**  
   - **Online** | Firmware: v2.1.7  
   - **Metrics:**  
     - Temperature: 15.73°C  
     - Network Latency: 412 ms  
     - Vibration: 3.667 mg  
     - Battery: 31%  
     - Signal: -68 dBm  

3. **2025-11-18 18:24:27**  
   - **Online** | Firmware: v3.9.9  
   - **Metrics:**  
     - Temperature: 15.53°C  
     - Network Latency: 442 ms  
     - Vibration: 4.224 mg  
     - Battery: 42%  
     - Signal: -59 dBm  

4. **2025-11-18 20:31:27**  
   - **Online** | Firmware: v3.3.2  
   - **Metrics:**  
     - Temperature: 43.87°C ⚠️  
     - Network Latency: 210 ms  
     - Vibration: 6.7